In [95]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import folium
from keras.preprocessing.sequence import TimeseriesGenerator

In [96]:
data = pd.read_csv("MOCK_DATA (3).csv",index_col='Date1',parse_dates=True)
data.index.freq = 'MS'

In [97]:
x = data.iloc[:,2:5]


In [98]:
kmeans = KMeans(4)
kmeans.fit(x) 

C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4)

In [118]:
identified_clusters = kmeans.fit_predict(x)


C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [100]:
data_with_clusters = data.copy()
data_with_clusters['Cluster'] = identified_clusters


In [127]:
xdata = data_with_clusters.sort_values(by="Cluster")
xdata = xdata.iloc[4:11,]

,Pickups,Time,Lattitude,Longitude,Cluster
Date1,,,,,
2022-07-01,44,23:28:45,12.985077,80.208673,1
2023-07-01,10,23:18:15,13.014259,80.222632,1
2021-06-01,11,14:35:58,12.996587,80.240711,1
2023-06-01,28,13:11:08,12.971601,80.235667,1
2022-11-01,26,2:43:55,12.991364,80.206460,1
2022-08-01,36,2:22:22,13.012365,80.202612,1
2021-12-01,17,1:14:53,13.001165,80.227373,1


In [134]:

map10 = folium.Map(location=[13.081689, 80.269032],tiles="Stamen Toner")
for i,j in xdata.iterrows():
    folium.Circle(
    radius=j['Pickups']*10,
    location=[j['Lattitude'],j['Longitude']],
    popup="Demand",
    color="crimson",
    fill=True,
).add_to(map10)


Training And Testing


In [103]:
train = data_with_clusters.iloc[:30]
test = data_with_clusters.iloc[0:6]
test


,Pickups,Time,Lattitude,Longitude,Cluster
Date1,,,,,
2021-01-01,48,1:47:14,12.904861,80.235158,2
2021-02-01,19,6:30:25,12.926894,80.198804,2
2021-03-01,20,9:06:50,12.941212,80.201115,2
2021-04-01,23,7:16:07,12.913970,80.232802,2
2021-05-01,20,18:10:45,13.048809,80.212946,3
2021-06-01,11,14:35:58,12.996587,80.240711,1


In [104]:
arr = [[12.99029982,80.22106985]]
trys = kmeans.predict(arr)
trys

C:\Users\user 2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


array([1])

In [105]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train1 = train.drop(['Time','Cluster','Pickups'],axis=1)
test1 = test.drop(['Time','Cluster','Pickups'],axis=1)

In [106]:
datra = scaler.fit(train1)
scaled_train = datra.transform(train1)
scaled_test = datra.transform(test1)
scaled_test

array([[0.06978319, 0.88489187],
       [0.16249983, 0.13132262],
       [0.22274541, 0.17920925],
       [0.10811448, 0.83604592],
       [0.6755035 , 0.42445927],
       [0.4557618 , 1.        ]])

In [107]:
n_input = 3
n_features = 2
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [108]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

Given the Array: 
[0.06978319 0.88489187 0.16249983 0.13132262 0.22274541 0.17920925]
Predict this y: 
 [[0.10811448 0.83604592]]


In [109]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [110]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [111]:
model.fit(generator,epochs=100)
model.input_shape


Epoch 1/100
27/27 [==============================] - 2s 2ms/step - loss: 0.1831
Epoch 2/100
27/27 [==============================] - 0s 3ms/step - loss: 0.1042
Epoch 3/100
27/27 [==============================] - 0s 4ms/step - loss: 0.0989
Epoch 4/100
27/27 [==============================] - 0s 4ms/step - loss: 0.0969
Epoch 5/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0957
Epoch 6/100
27/27 [==============================] - 0s 3ms/step - loss: 0.0939
Epoch 7/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0950
Epoch 8/100
27/27 [==============================] - 0s 3ms/step - loss: 0.0924
Epoch 9/100
27/27 [==============================] - 0s 3ms/step - loss: 0.0899
Epoch 10/100
27/27 [==============================] - 0s 4ms/step - loss: 0.0926
Epoch 11/100
27/27 [==============================] - 0s 3ms/step - loss: 0.0927
Epoch 12/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0892
Epoch 13/100
27/27 [=================

(None, 3, 2)

In [119]:
last_train_batch = scaled_test
last_train_batch = last_train_batch.reshape((2,3,n_features))


In [116]:
model.predict(last_train_batch)
x0 = [[0.4352999,0.7217424],[0.53012108, 0.62524348]]


1/1 [==============================] - 0s 49ms/step


In [117]:
scaler.inverse_transform(X=x0)

array([[12.99172476, 80.22728737],
       [13.0142588 , 80.22263209]])

In [ ]:

map9 = folium.Map(location=[13.081689, 80.269032],tiles="Stamen Toner")
for i,j in test.iterrows():
    folium.Marker(list((j['Lattitude'],j['Longitude']))).add_to(map9)


In [140]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="DemandPrediction")
location = geolocator.geocode("Perambur")
print((location.latitude, location.longitude))

(13.1121242, 80.245022)
